In [1]:
from mip import *
import datetime
import pandas as pd
import numpy as np
from EMS import *

## DayAhead PARAMETERS

In [2]:
PARAM_DayAhead = {}
# add length check with res & horizon
PARAM_DayAhead['Horizon'] = 3*24*60        # horizon to optimize (min)
PARAM_DayAhead['Resolution'] = 15    # sampling period(min)
PARAM_DayAhead['PV_capacity'] = 50   # (kw) PV sizing for this EMS
TOU = getBuySellrate(Resolution=PARAM_DayAhead['Resolution'],
                                    Horizon=PARAM_DayAhead['Horizon'],
                                    TOU_CHOICE='THcurrent',
                                    start_time=datetime.timedelta(minutes=0))
PARAM_DayAhead['Buy_rate'] = TOU['buy'].to_numpy()
PARAM_DayAhead['Sell_rate'] = TOU['sell'].to_numpy()

PARAM_DayAhead['battery'] = {}
PARAM_DayAhead['battery']['charge_effiency'] = [0.95,0.95];              #  bes charge eff
PARAM_DayAhead['battery']['discharge_effiency'] = [0.95*0.93,0.95*0.93]; #  bes discharge eff note inverter eff 0.93-0.96
PARAM_DayAhead['battery']['discharge_rate'] = [30,30]; # kW max discharge rate
PARAM_DayAhead['battery']['charge_rate'] = [30,30]; # kW max charge rate
PARAM_DayAhead['battery']['actual_capacity'] = [125,125]; # kWh soc_capacity 
PARAM_DayAhead['battery']['initial'] = [50,50]; # userdefined int 0-100 %
PARAM_DayAhead['battery']['min'] = [20,20]; #min soc userdefined int 0-100 %
PARAM_DayAhead['battery']['max'] = [80,80]; #max soc userdefined int 0-100 %
PARAM_DayAhead['battery']['num_batt'] = len(PARAM_DayAhead['battery']['actual_capacity'])

## Intraday PARAMETERS

In [3]:
PARAM_Intraday = {}
# add length check with res & horizon
PARAM_Intraday['Horizon'] = 1*60        # horizon to optimize (min)
PARAM_Intraday['Resolution'] = 5    # sampling period(min)
PARAM_Intraday['PV_capacity'] = 50   # (kw) PV sizing for this EMS
TOU = getBuySellrate(Resolution=PARAM_Intraday['Resolution'],
                                    Horizon=PARAM_Intraday['Horizon'],
                                    TOU_CHOICE='THcurrent',
                                    start_time=datetime.timedelta(minutes=0))
PARAM_Intraday['Buy_rate'] = TOU['buy'].to_numpy()
PARAM_Intraday['Sell_rate'] = TOU['sell'].to_numpy()
PARAM_Intraday['battery'] = {}
PARAM_Intraday['battery']['charge_effiency'] = [0.95,0.95];              #  bes charge eff
PARAM_Intraday['battery']['discharge_effiency'] = [0.95*0.93,0.95*0.93]; #  bes discharge eff note inverter eff 0.93-0.96
PARAM_Intraday['battery']['discharge_rate'] = [30,30]; # kW max discharge rate
PARAM_Intraday['battery']['charge_rate'] = [30,30]; # kW max charge rate
PARAM_Intraday['battery']['actual_capacity'] = [125,125]; # kWh soc_capacity 
PARAM_Intraday['battery']['initial'] = [50,50]; # userdefined int 0-100 %
PARAM_Intraday['battery']['min'] = [20,20]; #min soc userdefined int 0-100 %
PARAM_Intraday['battery']['max'] = [80,80]; #max soc userdefined int 0-100 %
PARAM_Intraday['battery']['num_batt'] = len(PARAM_Intraday['battery']['actual_capacity'])

In [4]:
root_folder = 'predict/'
load_dayAhead = pd.read_csv(root_folder + 'load_Dayahead_20230302_20231228.csv',parse_dates=['ds'])
pv_dayahead = pd.read_csv(root_folder + 'pv_Dayahead_20230102_20231228.csv',parse_dates=['ds'])
load_intraday = pd.read_csv(root_folder + 'load_Intraday_20230301_20231230.csv',parse_dates=['ds'])
pv_intraday = pd.read_csv(root_folder + 'pv_Intraday_20230101_20231231.csv',parse_dates=['ds'])
pv_scaling_factor = 50/8 # scale PV from 8 kW to 50 kW conversion factor conversion kW/(W/m^2)
# scale PV and PV >= 0
pv_dayahead.iloc[:,1:] = pv_dayahead.iloc[:,1:].clip(lower=0)*pv_scaling_factor
pv_intraday.iloc[:,1:] = pv_intraday.iloc[:,1:].clip(lower=0)*pv_scaling_factor
# load must >= 0
load_dayAhead.iloc[:,1:] = load_dayAhead.iloc[:,1:].clip(lower=0)
load_intraday.iloc[:,1:] = load_intraday.iloc[:,1:].clip(lower=0)




## Rolling

In [6]:
Num_days = 2
initial_date = pd.to_datetime('2023-11-01')
initial_time = pd.to_datetime('2023-11-01')
dayAhead_sol_list = pd.DataFrame()
IntraDay_sol_list = pd.DataFrame()
Resolution_Intraday_HR = PARAM_Intraday['Resolution']/60
Resolution_dayAhead_HR = PARAM_DayAhead['Resolution']/60
First_time_init = True # boolean for checking if it is first day of rolling
for day in range(Num_days) :
    PARAM_DayAhead['Start_date'] = initial_date + pd.Timedelta(days=day)
    PARAM_DayAhead['PV'] = pv_dayahead[pv_dayahead['ds'] == PARAM_DayAhead['Start_date'] ].iloc[:,1:].to_numpy().flatten()
    PARAM_DayAhead['PL'] = load_dayAhead[load_dayAhead['ds'] == PARAM_DayAhead['Start_date']].iloc[:,1:].to_numpy().flatten()    
    # set initial soc
    if day == 0:
        PARAM_DayAhead['battery']['initial'] = [50,50]
    else :
                       
        PARAM_DayAhead['battery']['initial'] = [dayAhead_sol.iloc[96,:][f'soc_{j}'] for j in range(PARAM_DayAhead['battery']['num_batt'])]
        
    #get DA sol
    print(f'begin optimizing day{day}')
    dayAhead_sol = EMS_1_opt(PARAM_DayAhead,energycost=1,multibatt=1,chargebatt=1,smoothcharge=0.3)
    dayAhead_sol_list = pd.concat([dayAhead_sol_list,dayAhead_sol.iloc[:96,:]])
    print(f'finish optimizing day{day}')
    # rolling 5 minute 
    for i in range(288) :
        PARAM_Intraday['Start_date'] = PARAM_DayAhead['Start_date'] + pd.Timedelta(minutes=5*i)
        PARAM_Intraday['End_date'] = PARAM_Intraday['Start_date'] + pd.Timedelta(minutes=PARAM_Intraday['Horizon'])
        PARAM_Intraday['PV'] = pv_intraday[pv_intraday['ds'] == PARAM_Intraday['Start_date'] ].iloc[:,1:].to_numpy().flatten()
        PARAM_Intraday['PL'] = load_intraday[load_intraday['ds'] == PARAM_Intraday['Start_date'] ].iloc[:,1:].to_numpy().flatten()
        windowed_dayAhead_plan = dayAhead_sol[ (dayAhead_sol['datetime'] >= PARAM_Intraday['Start_date'] - pd.Timedelta(minutes=10)) & (dayAhead_sol['datetime'] < PARAM_Intraday['End_date'])]
        # use DA solution as parameters for HA
       
        PARAM_Intraday['Pchg'] = windowed_dayAhead_plan['Pchg_0'].to_numpy().flatten()
        PARAM_Intraday['Pdchg'] = windowed_dayAhead_plan['Pdchg_0'].to_numpy().flatten()
        PARAM_Intraday['Pnet'] = windowed_dayAhead_plan['Pnet'].to_numpy().flatten()
        TOU = getBuySellrate(Resolution=PARAM_Intraday['Resolution'],
                                    Horizon=PARAM_Intraday['Horizon'],
                                    TOU_CHOICE='THcurrent',
                                    start_time=datetime.timedelta(minutes=5*i))
        PARAM_Intraday['Buy_rate'] = TOU['buy'].to_numpy()
        PARAM_Intraday['Sell_rate'] = TOU['sell'].to_numpy()
        #set initial soc
        if First_time_init:
            PARAM_Intraday['battery']['initial'] = [50,50]
            First_time_init = False
        else :
                       
            PARAM_Intraday['battery']['initial'] = [intraday_sol.iloc[1,:][f'soc_{j}'] for j in range(PARAM_Intraday['battery']['num_batt'])]
        print(f'begin optimizing day {day} step {i}')
        # get HA sol
        intraday_sol = EMS_rolling(PARAM_Intraday,minute=5*i,energycost=1,Pnet_diff=0.1,Pchg_diff=0.3,Pdchg_diff=0.2,chargebatt=0.07,smoothcharge=0.06)
        # if i < 276 :
        #     IntraDay_sol_list = pd.concat([IntraDay_sol_list,intraday_sol.iloc[0:1,:]])
        # else :
        #     IntraDay_sol_list = pd.concat([IntraDay_sol_list,intraday_sol])
        IntraDay_sol_list = pd.concat([IntraDay_sol_list,intraday_sol.iloc[0:1,:]])
        print(f'finish optimizing day {day} step {i}')


    
    
    
    

begin optimizing day0
finish optimizing day0
begin optimizing day 0 step 0
finish optimizing day 0 step 0
begin optimizing day 0 step 1
finish optimizing day 0 step 1
begin optimizing day 0 step 2
finish optimizing day 0 step 2
begin optimizing day 0 step 3
finish optimizing day 0 step 3
begin optimizing day 0 step 4
finish optimizing day 0 step 4
begin optimizing day 0 step 5
finish optimizing day 0 step 5
begin optimizing day 0 step 6
finish optimizing day 0 step 6
begin optimizing day 0 step 7
finish optimizing day 0 step 7
begin optimizing day 0 step 8
finish optimizing day 0 step 8
begin optimizing day 0 step 9
finish optimizing day 0 step 9
begin optimizing day 0 step 10
finish optimizing day 0 step 10
begin optimizing day 0 step 11
finish optimizing day 0 step 11
begin optimizing day 0 step 12
finish optimizing day 0 step 12
begin optimizing day 0 step 13
finish optimizing day 0 step 13
begin optimizing day 0 step 14
finish optimizing day 0 step 14
begin optimizing day 0 step 15

In [7]:
dayAhead_sol

,datetime,PARAM_PV,PARAM_PL,Buy_rate,Sell_rate,Pnet,u1,s1,Pchg_0,Pdchg_0,...,soc_0,upper_bound_Pchg_0,upper_bound_Pdchg_0,Pchg_1,Pdchg_1,xchg_1,xdchg_1,soc_1,upper_bound_Pchg_1,upper_bound_Pdchg_1
0,2023-11-02 00:00:00,0.000000,1.603806,2.6,2,-15.460540,10.049351,0.0,6.928367,0.000000,...,57.598896,0.0,0.0,6.928367,0.000000,1.0,0.0,57.598896,0.000000e+00,0.0
1,2023-11-02 00:15:00,0.035633,1.563998,2.6,2,-15.385099,10.000314,0.0,6.928367,0.000000,...,58.915286,0.0,0.0,6.928367,0.000000,1.0,0.0,58.915286,0.000000e+00,0.0
2,2023-11-02 00:30:00,0.000000,1.604293,2.6,2,-15.461027,10.049667,0.0,6.928367,0.000000,...,60.231675,0.0,0.0,6.928367,0.000000,1.0,0.0,60.231675,-6.558541e-15,0.0
3,2023-11-02 00:45:00,0.000000,1.757984,2.6,2,-15.614717,10.149566,0.0,6.928367,0.000000,...,61.548065,0.0,0.0,6.928367,0.000000,1.0,0.0,61.548065,0.000000e+00,0.0
4,2023-11-02 01:00:00,0.000000,1.796060,2.6,2,-15.652793,10.174316,0.0,6.928367,0.000000,...,62.864455,0.0,0.0,6.928367,0.000000,1.0,0.0,62.864455,0.000000e+00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,2023-11-04 22:45:00,0.074331,0.285654,5.8,2,1.550068,0.000000,0.0,0.000000,0.880696,...,42.588161,0.0,0.0,0.000000,0.880696,0.0,1.0,42.588161,0.000000e+00,0.0
284,2023-11-04 23:00:00,0.000000,0.149121,5.8,2,1.612270,0.000000,0.0,0.000000,0.880696,...,42.388796,0.0,0.0,0.000000,0.880696,0.0,1.0,42.388796,0.000000e+00,0.0
285,2023-11-04 23:15:00,0.000000,0.102497,2.6,2,1.658895,0.000000,0.0,0.000000,0.880696,...,42.189431,0.0,0.0,0.000000,0.880696,0.0,1.0,42.189431,0.000000e+00,0.0
286,2023-11-04 23:30:00,0.121094,0.000000,2.6,2,1.882485,0.000000,0.0,0.000000,0.880696,...,41.990066,0.0,0.0,0.000000,0.880696,0.0,1.0,41.990066,0.000000e+00,0.0


In [8]:
intraday_sol

,datetime,PARAM_PV,PARAM_PL,Buy_rate,Sell_rate,Pnet,u1,s1,Pchg_0,Pdchg_0,...,Pchg_1,Pdchg_1,xchg_1,xdchg_1,soc_1,upper_bound_Pchg_1,upper_bound_Pdchg_1,uPnet,uPchg,uPdchg
0,2023-11-02 23:55:00,0.038754,3.554532,2.6,2,-3.515777,0.761752,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.398542,0.0,0.0
1,2023-11-03 00:00:00,0.016094,3.874228,2.6,2,-3.858133,0.835929,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.740898,0.0,0.0
2,2023-11-03 00:05:00,0.009124,3.612093,2.6,2,-3.602969,0.780643,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.485734,0.0,0.0
3,2023-11-03 00:10:00,0.000000,3.564254,2.6,2,-3.564254,0.772255,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.560822,0.0,0.0
4,2023-11-03 00:15:00,0.000000,3.663119,2.6,2,-3.663119,0.793676,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.659687,0.0,0.0
5,2023-11-03 00:20:00,0.000000,3.334619,2.6,2,-3.334619,0.722501,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.331187,0.0,0.0
6,2023-11-03 00:25:00,0.000000,3.552549,2.6,2,-3.552549,0.769719,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.599130,0.0,0.0
7,2023-11-03 00:30:00,0.000000,3.480986,2.6,2,-3.480986,0.754214,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.527566,0.0,0.0
8,2023-11-03 00:35:00,0.000000,3.640088,2.6,2,-3.640088,0.788686,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.686669,0.0,0.0
9,2023-11-03 00:40:00,0.000000,3.483789,2.6,2,-3.483789,0.754821,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.633234,0.0,0.0


In [9]:
#dayAhead_sol_list.to_csv('dayAhead_sol_20231128_20231130.csv',index=False)
#dayAhead_sol_list.to_csv('dayAhead_sol_20231101_20231104.csv',index=False)

In [10]:
#IntraDay_sol_list.to_csv('Intraday_sol_20231128_20231130.csv',index=False)
#IntraDay_sol_list.to_csv('Intraday_sol_20231101_20231104.csv',index=False)

In [11]:
#dayAhead_sol_list = pd.read_csv('dayAhead_sol_NOV_2023_nolimit.csv',parse_dates=['datetime'])
#IntraDay_sol_list = pd.read_csv('Intraday_sol_NOV_2023_nolimit.csv',parse_dates=['datetime'])

In [12]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=2, cols=1,subplot_titles=('Solar generation','Load consumption'))

fig.add_trace(
    go.Scatter(x=IntraDay_sol_list['datetime'], y=IntraDay_sol_list['PARAM_PV'],name='Predicted PV intraday'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=dayAhead_sol_list['datetime'], y=dayAhead_sol_list['PARAM_PV'],name='Predicted PV dayAhead'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=IntraDay_sol_list['datetime'], y=IntraDay_sol_list['PARAM_PL'],name='Predicted load intraday'),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=dayAhead_sol_list['datetime'], y=dayAhead_sol_list['PARAM_PL'],name='Predicted load dayAhead'),
    row=2, col=1
)
fig.update_yaxes(title_text = 'Power (kW)',row=1,col=1)
fig.update_yaxes(title_text = 'Power (kW)',row=2,col=1)
fig.update_layout(height=600, width=800, title_text="Predicted PV and Load")
fig.show()

In [13]:
intraday_sol[['uPnet','uPchg','uPdchg']].sum(axis=0)

uPnet     6.791288
uPchg     0.000000
uPdchg    0.000000
dtype: float64

In [14]:
dayAhead_sol_list[dayAhead_sol_list['datetime'] == '2023-11-01 23:45:00']

,datetime,PARAM_PV,PARAM_PL,Buy_rate,Sell_rate,Pnet,u1,s1,Pchg_0,Pdchg_0,...,soc_0,upper_bound_Pchg_0,upper_bound_Pdchg_0,Pchg_1,Pdchg_1,xchg_1,xdchg_1,soc_1,upper_bound_Pchg_1,upper_bound_Pdchg_1
95,2023-11-01 23:45:00,0.059217,2.934845,2.6,2,-4.265989,2.772893,0.0,0.695181,0.0,...,57.466812,0.0,0.0,0.695181,0.0,1.0,0.0,57.466812,0.0,0.0


In [15]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=2, cols=2,subplot_titles=('State of Charge (SoC)','Charge power','Expense with EMS 1','Discharge power'),specs=[[{"secondary_y": True}, {"secondary_y": True}],
                           [{"secondary_y": True}, {"secondary_y": True}]])
excess_gen_DA = dayAhead_sol_list['PARAM_PV'] - dayAhead_sol_list['PARAM_PL']
excess_gen_HA = IntraDay_sol_list['PARAM_PV'] - IntraDay_sol_list['PARAM_PL']
cum_expense_DA = dayAhead_sol_list['u1'].cumsum()
cum_expense_HA = IntraDay_sol_list['u1'].cumsum()
# plot(1,1)
fig.add_trace(
    go.Scatter(x=IntraDay_sol_list['datetime'], y=excess_gen_HA, name = 'Excess gen HA',mode='lines+markers',line={'color':'#000000'}),
    row=1, col=1,secondary_y=True
)
fig.add_trace(
    go.Scatter(x=dayAhead_sol_list['datetime'], y=excess_gen_DA,name = 'Excess gen DA',mode='lines+markers',line={'color':'#FF0000'}),
    row=1, col=1,secondary_y=True
)
fig.add_trace(
    go.Scatter(x=IntraDay_sol_list['datetime'], y=IntraDay_sol_list['soc_0'], name = 'SoC HA',line={'color':'#000000'}),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=dayAhead_sol_list['datetime'], y=dayAhead_sol_list['soc_0'],name = 'SoC DA',line={'color':'#FF0000'}),
    row=1, col=1
)

# plot (1,2)
fig.add_trace(
    go.Scatter(x=IntraDay_sol_list['datetime'], y=IntraDay_sol_list['Pchg_0'], name = 'Pchg HA',line={'color':'#000000'}),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=dayAhead_sol_list['datetime'], y=dayAhead_sol_list['Pchg_0'],name = 'Pchg DA',mode='markers',line={'color':'#FF0000'}),
    row=1, col=2
)
# plot (2,1)
fig.add_trace(
    go.Scatter(x=IntraDay_sol_list['datetime'], y=cum_expense_HA, name = 'Cumulative expense HA',line={'color':'#000000'}),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=dayAhead_sol_list['datetime'], y=cum_expense_DA,name = 'Cumulative expense DA',line={'color':'#FF0000'}),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=IntraDay_sol_list['datetime'], y=IntraDay_sol_list['Pnet'], name = 'Pnet HA',mode='lines+markers',line={'color':'#000000'}),
    row=2, col=1, secondary_y=True
)
fig.add_trace(
    go.Scatter(x=dayAhead_sol_list['datetime'], y=dayAhead_sol_list['Pnet'],name = 'Pnet DA',mode='lines+markers',line={'color':'#FF0000'}),
    row=2, col=1, secondary_y=True
)

# plot (2,2)
fig.add_trace(
    go.Scatter(x=IntraDay_sol_list['datetime'], y=IntraDay_sol_list['Pdchg_0'], name = 'Pdchg HA',line={'color':'#000000'}),
    row=2, col=2
)
fig.add_trace(
    go.Scatter(x=dayAhead_sol_list['datetime'], y=dayAhead_sol_list['Pdchg_0'],name = 'Pdchg DA',mode='markers',line={'color':'#FF0000'}),
    row=2, col=2
)

fig.update_yaxes(title_text='SoC (%)',row=1,col=1)
fig.update_yaxes(title_text='Excess gen (kW)',row=1,col=1,secondary_y=True)
fig.update_yaxes(title_text='Expense (THB)',row=2,col=1)
fig.update_yaxes(title_text='Pnet (kW)',row=2,col=1,secondary_y=True)
fig.update_yaxes(title_text='Power (kW)',row=1,col=2)
fig.update_yaxes(title_text='Power (kW)',row=2,col=2)
fig.update_layout(height=1000, width=1500, title_text="")
fig.show()

In [16]:
test.shape

NameError: name 'test' is not defined